How to prune neurons?
- Maybe by replace activations with zero?
- Maybe with replace activations with random? What is random?

In [1]:
random_input = """
3KhCfVVkC5VNSchVtAyKurMGCHaMHS25
SSKrJRD6yaNZSwSN3MsVBRNs6KmvQG3q
YFntXDqxWtBKDcNHcDmHuRbNa39qWc7w
hK2LZPXKxUP8qWfmKfCEyfEzcUWJMDca
Hve9qM7BGfurBeNCDryd7Ad2Rc2VYxym
X4DGeuJZ8Thq4C3jjUAeAPpCn7wQZVeL
x9A35JypdKTvgYGQKvfD6bgK3hVfq8tH
hnDUWuTQppnBn4VTMFhYTK9A5dLHyqp9
p5uCpD4uBzFJH2Yx4GGg5uwt6uCuVS2P
fD239awAbXRh7euz3EzFVBC7eeSmeMz2
Np5YEW8Ra5x26pqQvw897CRJjbmm3RxN
cmYJqUFFwRPVAQrbVTHxfPQtf4R9NCJz
fRGHRfNcQ9ZD9UapmNmfcXEKre8sBtyD
nSpjET6nhDnqfdC79Qd4mSEQnSNyk2FK
PkRbd4mpgyahgNTJh4ZcWY8SpVab8jMv
BTA9yFmKJmtKTzcpvgdyy7HbbAyDZRzt
CuC5Jw5VLExnJgLX7Wath49LcL4qEJmC
7w335MF9hGaYY9s7nQwbAzpARReC3Edf
jVzxd7f2ru64f8Zb8KJr6JeUprrqyaQz
fm9wtL2hFkxprUc5vGnJP3fghyNNeUFS
3bsDEnbFVJaYGYnQBWeLtm74Fp4HMyhQ
YFXeVY4EFwSYMRCCLdd3pACADcZ5LjSr
BTR9ryHMACxU5uhM7JTGRW9sAzzfM26W
2vzuvGWjTFDsVfYWrAzvnm56C5wstVeT
y9sEexYkCwag3MEGxr3qnJpQYESPZuMt
EhhjeFTBnaMaZ2HJNHfEj2WkJXNGPjF4
qsv48r3tEk462nDQ7UhTRcZvnDwAMSgF
tv6PX2HXz8EqzCv2jgYUFvgajXjfmK9k
zy9bjLaSyG6qSrSMWFWhLkwLWrednXdW
FE822FCqNK8vsCxtpWpVCYK7C42gy7DF
mFffPa53XWdT69UNefXumy5FGtqRasfr
sA3yNr96uRSnBsjTSDyA7uLfxeWCcu87
w7MXtQVXuNR9g9EXmNmLUR5nEWPjVsf2
C8AdARb5q4VqafTZMYhxBUFcp2DNQf6Z
7kFrJrC9kU4Lrmt4TUhr93EN28Wx4Mr6
pGn8bqfvttLYW7KpWFxMwjHfnreh53ZB
MHG477QV9qYmXYweN5uXe4Zd2Q94mGFu
w9B2uzLsSqMGq9ZUGAfGtbmzn3ybn6dC
bDwdQ8XhkBxJSHfTHxncNLEdPPkKvsjR
RNUcBELxE48ChL36fUWWWcJtxYCuA9zv
cMhFPHD8MBY3cX4qwN8vpcRCADM7g9px
2J2ceK9MVTXEdJd93haQZG6E8eGLgbWt
27jjR7rXYAaqm9uDuVLMgDLK9uMNhjAt
sY89ymz9hvPeM7bGYKvCwfb8GujfKQAV
w2JmRFHUQP4hBfDT9ccSG9Wd8mQUuGmV
YVpEbUZfRQZ8wmqPkGQjSZZDU9XLWSzy
NAgvZHXvpCbBYFzWtQPefsVsC6sD64Ta
qyHRmTxftS24QWmZzVMcemUhbfgZTxqu
kg7CGfNL4S4cHPFhV7n2hBxDXDNLRyYM
4ZAQX5m9phCqPnTQw8T9wKwEhnm2pRLw
2D748S89SSv5LYLUMXeVeG6EAZRtuyuD
QbPJvRF3EK6zvJvqV5QtWVpkveQMU6xq
757F97h5Af8vmM58t7k9U6GAr4RvFFZ6
WnYkQVXE74MXw4HLWJwnCu3YevDaSNNJ
EeFqkmEGk3wnkrVyN8AKShTBLcrtw7qa
WQktwdF9qRqF2Zf58ZC3suUg9B76TPD9
nh3fFpMUuSQAwYzF8CH4zzRD2tV3F5zj
xEKmnCpxdERCKgRSYX6KWRrnFwhuEp25
vQaYUp4mCnXbgd6AVkUzVv64JnbC7Yxp
23gpv5NYp6vfJCFuzRwaMbE6RRurKHP6
FQ9j9KbSTx3CmgThn4Zfv39q2u6AfrBW
m5sPpQQeh666FqFxGfAq7ttA8x8JPSMH
5dh7EkjFhhxrQRBWMrSCHLMM3b9TzXKK
ANm2BruU43uW397dKw9vytSkbxS9AkAN
6rbAxuY96SdwfN4vLHm7hU7nEDvkWt7S
n7jETEfSRTj9upFgxwKpn3nHkZ43zYCz
WdthZRfN5m5WwdNYDgMbjAvZNCjgUyC6
FT9nhMhJ7XChJG5q4u7Aa2PUMF6QNrqa
MmuAEnZNdPCq7544ZdaW9vej59GqhUJn
KHU9pfSwKbRt4wYV4de6JwmMkAFuawbn
FXkYyCDCfmJAt3fJDNyuZpzXbPxreHgC
Q6dSMqrD6P4w8nUK99eFHeLfhdUCy4Aa
fDskbZrqb7P4ZWqMVPL8K2HqfEYSYMrf
sXnyZ87RQ3fxSLMfym5SyzX9E8x3zKsv
rRTWpGCbrYgVf3GszHUkCGuSkpfPW3xH
krknS73Rzt4DWjZTpx63C8zTPHyyuVTG
qSFHkwa9McvSPtmK8v9VeU7HydWEdvxk
xLvPhhYa2QwUqCQut4gRSXTjfRWv8r4e
jkq3vwrjnEJDv2HNEf7yqm3gVR2Hq7Zr
vm35QLghBDsKxJXEM9S8wFNrVTUAvjs9
muaD42eDfqn6uCNKrYPTw44Q9gzHW5mE
WfEtEWk2wj26FAzxZMmnvxuFsU3Yucdb
H2vXqB86dUWRucMqcjfg8FDBMDwdmcUr
ENNTUEdPtwhqSRN5hyWKPjGTxQSzuCx9
VSpjn2k6LFzE3YKTLxXNz4UgTM4X98CD
Zkat6pFXwudGUucLFQgsh95g4BjbKTTH
vCUfMfQhB8Qxuz2MkGtKSbAfNLFYd65K
y6fq6hkHqQZHFYFKu6jDdU3ZJup9zUSK
49gDBDZnqQGgSraZkUKnTEdwVqAWaTWN
D28nXSCmzvUUYvmrYqZR6fVqqk7dcEFN
duksGqqce8LXRhzesSGCmunpGWY4ewQY
afK79yU7GMm8ZFnaZ4Ur2xA2PfDksu2M
PJhv5X2fyNwRQPHhZjsQJJtedcx5Ktt7
kuqYWxwt5SP5CpAbLNr3V4xGmMMCsrEr
rnXsS37decqMxW2TKxrTNJDUxvuMPaXA
Z3zVRbVa72DQ2vcNBKx423Zn3GXcqUhk
zaKEXhY79w4D7ejMrHhNeG7HJfwHzPDV
pG5SQPZPK37AWHeu2rNjkArVYLRNXepD
fwPhRX2jCKWSTQMxwcswNJEcsMhA8HEh
vBpRUMktY9dh7mZpAfkT5VyVXSQh3Dns
"""

In [2]:
from taker import Model

m = Model("facebook/galactica-125m", limit=1000)

print(m.masks["mlp_pre_out"].state_dict().keys())

/home/ubuntu/taker/src/taker/model.py:143: UserWarning: Model facebook/galactica-125m not tested.
  warnings.warn( f"Model {model_repo} not tested." )


config.json:   0%|          | 0.00/787 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/166 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.14M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/250M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

- Loaded facebook/galactica-125m
 - Registered 12 Attention Layers
odict_keys(['layers.0.param', 'layers.0.offset', 'layers.1.param', 'layers.1.offset', 'layers.2.param', 'layers.2.offset', 'layers.3.param', 'layers.3.offset', 'layers.4.param', 'layers.4.offset', 'layers.5.param', 'layers.5.offset', 'layers.6.param', 'layers.6.offset', 'layers.7.param', 'layers.7.offset', 'layers.8.param', 'layers.8.offset', 'layers.9.param', 'layers.9.offset', 'layers.10.param', 'layers.10.offset', 'layers.11.param', 'layers.11.offset'])


In [3]:
apo_rand = m.get_attn_pre_out_activations(random_input)
print(apo_rand.shape)

torch.Size([12, 1000, 12, 64])


In [4]:
from taker.eval import evaluate_all

evaluate_all(m, 1e5, ["pile"])

100000.0


  0%|          | 0/100000.0 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

Acc: 76.43|49.73 (Skip: 69.67|43.13): : 100410it [05:30, 303.92it/s]                           


{'loss_data': {'pile': {'perplexity': nan,
   'loss': 2.6816,
   'log_loss': -0.5583}},
 'accuracy': {'pile': {'base': 49.72670359662481,
   'topk': 76.433978504116,
   'skip': 43.13215815157853,
   'topk_skip': 69.67433522557515}},
 'misc': {'pile': {'accuracy_data': {'num_predictions': 135933,
    'num_accurate': 67595,
    'num_topk_accurate': 103899,
    'num_skip_predictions': 100410,
    'num_skip_accurate': 43309,
    'num_topk_skip_accurate': 69960,
    'token_counts': array([0., 0., 0., ..., 0., 0., 0.])},
   'eval_config': {'dataset_name': 'pile',
    'dataset_repo': 'monology/pile-uncopyrighted',
    'dataset_subset': None,
    'dataset_type': 'prediction',
    'dataset_text_key': 'text',
    'dataset_text_label_key': 'label',
    'dataset_filter': None,
    'dataset_has_test_split': True,
    'dataset_split': None,
    'streaming': True,
    'sample_size': 100000.0,
    'skip_token_strings': ['The',
     'are',
     'it',
     '\\',
     '*',
     '-',
     '  ',
     'by',

In [7]:
from taker.prune import prune_random

ff_pruned, attn_pruned = prune_random(m, attn_frac=0.5, ff_frac=0.0)

/home/ubuntu/taker/src/taker/model.py:1018: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  remove_indices = torch.tensor(remove_indices).to(self.device)


ValueError: too many values to unpack (expected 2)

In [16]:
import torch
vec = torch.ones([m.cfg.n_layers, 2, m.cfg.d_model]).to(m.device)
print(m.run_masking(vec, "attn_pre_out"))
print(m.run_inverse_masking(vec, "attn_pre_out"))

tensor([[[1., 1., 0.,  ..., 0., 0., 1.],
         [1., 1., 0.,  ..., 0., 0., 1.]],

        [[1., 1., 1.,  ..., 0., 1., 1.],
         [1., 1., 1.,  ..., 0., 1., 1.]],

        [[0., 1., 0.,  ..., 1., 0., 1.],
         [0., 1., 0.,  ..., 1., 0., 1.]],

        ...,

        [[1., 1., 1.,  ..., 0., 0., 1.],
         [1., 1., 1.,  ..., 0., 0., 1.]],

        [[0., 1., 0.,  ..., 0., 1., 1.],
         [0., 1., 0.,  ..., 0., 1., 1.]],

        [[0., 1., 0.,  ..., 1., 1., 0.],
         [0., 1., 0.,  ..., 1., 1., 0.]]], device='cuda:0',
       grad_fn=<StackBackward0>)
tensor([[[0., 0., 1.,  ..., 1., 1., 0.],
         [0., 0., 1.,  ..., 1., 1., 0.]],

        [[0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 0.,  ..., 1., 0., 0.]],

        [[1., 0., 1.,  ..., 0., 1., 0.],
         [1., 0., 1.,  ..., 0., 1., 0.]],

        ...,

        [[0., 0., 0.,  ..., 1., 1., 0.],
         [0., 0., 0.,  ..., 1., 1., 0.]],

        [[1., 0., 1.,  ..., 1., 0., 0.],
         [1., 0., 1.,  ..., 1., 0., 0.]],

In [14]:
evaluate_all(m, 1e5, ["pile"])

100000.0


  0%|          | 0/100000.0 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

Acc: 67.55|37.63 (Skip: 58.61|31.05): : 100410it [05:32, 301.89it/s]                           


{'loss_data': {'pile': {'perplexity': nan,
   'loss': 3.5602,
   'log_loss': 0.4922}},
 'accuracy': {'pile': {'base': 37.632510133668795,
   'topk': 67.55239713682477,
   'skip': 31.046708495169803,
   'topk_skip': 58.60770839557813}},
 'misc': {'pile': {'accuracy_data': {'num_predictions': 135933,
    'num_accurate': 51155,
    'num_topk_accurate': 91826,
    'num_skip_predictions': 100410,
    'num_skip_accurate': 31174,
    'num_topk_skip_accurate': 58848,
    'token_counts': array([0., 0., 0., ..., 0., 0., 0.])},
   'eval_config': {'dataset_name': 'pile',
    'dataset_repo': 'monology/pile-uncopyrighted',
    'dataset_subset': None,
    'dataset_type': 'prediction',
    'dataset_text_key': 'text',
    'dataset_text_label_key': 'label',
    'dataset_filter': None,
    'dataset_has_test_split': True,
    'dataset_split': None,
    'streaming': True,
    'sample_size': 100000.0,
    'skip_token_strings': ['The',
     'are',
     'it',
     '\\',
     '*',
     '-',
     '  ',
     'by

In [18]:
apo_rand_masked = m.run_inverse_masking(apo_rand, "attn_pre_out")
print(apo_rand_masked.shape)

torch.Size([12, 1000, 12, 64])


In [30]:
m.update_actadd(apo_rand_masked.reshape(12, 1000, m.cfg.d_model), "attn_pre_out")

In [31]:
evaluate_all(m, 1e5, ["pile"])

100000.0


Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

Acc: 57.10|26.39 (Skip: 46.56|19.47): : 100410it [06:05, 274.54it/s]


{'loss_data': {'pile': {'perplexity': nan,
   'loss': 4.4804,
   'log_loss': 1.0563}},
 'accuracy': {'pile': {'base': 26.386528657500385,
   'topk': 57.09651078104655,
   'skip': 19.465192709889454,
   'topk_skip': 46.556119908375656}},
 'misc': {'pile': {'accuracy_data': {'num_predictions': 135933,
    'num_accurate': 35868,
    'num_topk_accurate': 77613,
    'num_skip_predictions': 100410,
    'num_skip_accurate': 19545,
    'num_topk_skip_accurate': 46747,
    'token_counts': array([0., 0., 0., ..., 0., 0., 0.])},
   'eval_config': {'dataset_name': 'pile',
    'dataset_repo': 'monology/pile-uncopyrighted',
    'dataset_subset': None,
    'dataset_type': 'prediction',
    'dataset_text_key': 'text',
    'dataset_text_label_key': 'label',
    'dataset_filter': None,
    'dataset_has_test_split': True,
    'dataset_split': None,
    'streaming': True,
    'sample_size': 100000.0,
    'skip_token_strings': ['The',
     'are',
     'it',
     '\\',
     '*',
     '-',
     '  ',
     'b